In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

def xml2df(xmlfile):
    """
    설명: mybatis XML 을 parsing해서 dataframe에 저장하는 함수
    ex) xml2df('MapperMng_ora2pg_tedkim.xml')
    """
    root = ET.parse(xmlfile).getroot()

    tags = {"tags":[], "sql_encore":[], "tag" :[]}
    for elem in root:
        tag = {}    

        for a in elem.attrib.keys():
            tag[a] = elem.attrib[a]    
        tags["tags"].append(tag)

        tags['sql_encore'].append(elem.text)
        tags['tag'].append(elem.tag)

    df_xml = pd.DataFrame(tags['tags'])
    df_xml.insert(0, 'tag', tags['tag'])
    df_xml['sql_encore'] = (tags['sql_encore'])

    # set dataframe index from 1
    df_xml.index = np.arange(1,len(df_xml)+1)

    #nana to null
    df_xml = df_xml.where(pd.notnull(df_xml), None)
    
    return df_xml.head()


In [3]:
xml2df('mappers/UserMapper.xml')

,tag,id,resultType,sql_encore
1,select,login,UserVO,"\n\tselect \n\t\tuid, upw, uname \n\t from\n\t..."
2,update,keepLogin,None,\n\tupdate tbl_user \n\t set \n\t \tsessionKey...
3,select,checkUserWithSessionKey,UserVO,\n\tselect * from tbl_user \n\t where \n\t \ts...
4,insert,register,None,"\n insert into tbl_user (uid, upw, uname) \..."
5,select,idCheck,int,\n\tSELECT COUNT(*) FROM tbl_user\n\tWHERE uid...


In [4]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

def get_pgsql(conn, sql):
    """
    설명: DB에 연결해서 결과를 출력할 수 있는 함수
    ex) result = get_pgsql(conn, sql)
    """
    
    print("sql" + sql)
    
    result = pd.read_sql_query(sql, conn)
    conn.close()

    return result 

In [5]:
conn = pg.connect("host='localhost' dbname='dataware_test' user='postgres' password='postgres'")
sql = """
          SELECT table_catalog,table_name 
              FROM information_schema.tables
    """

result = get_pgsql(conn, sql)
result.head()

sql
          SELECT table_catalog,table_name 
              FROM information_schema.tables
    


,table_catalog,table_name
0,dataware_test,md_mview_log
1,dataware_test,std_term_anly_words
2,dataware_test,imsi_md_mdl_chg_req_udp
3,dataware_test,tmd_df
4,dataware_test,tmd_mview
